In [389]:
import numpy as np
from numpy import sort
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import pprint
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, RobustScaler
%matplotlib inline

from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

In [410]:
FULL_DATASET = 'FULL TRAINING SET updated.csv'

### Feature Selection

In [416]:
df = pd.read_csv(FULL_DATASET)
df = df.drop(['Team1', 'Team2', 'GAMES', 'Seed1', 'Seed2', 'Round'], axis=1)
#df['Seed_Diff'] = df['Seed_Diff'].astype('category')
#df['Round'] = df['Round'].astype('category')

df.head()

,Result,Seed_Diff,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,1,-15,7.6847,-120,6.6882,-134,14.453,-183,21.6693,-235,...,-150,-195,-91,-49,-197,-139,-181,-180,-35,-268
1,1,-13,11.7930,-309,9.9867,-301,6.174,-64,9.4790,-82,...,-300,-167,-106,113,-121,255,-242,-76,-66,-225
2,1,-11,-5.4011,151,-5.5255,142,-3.461,45,3.2230,-38,...,141,-67,-116,181,-267,31,-154,32,-177,-206
3,1,-9,-4.4331,44,-3.1859,38,10.226,-148,16.5788,-215,...,33,-200,-172,-78,-143,-282,119,-149,26,-149
4,1,-7,8.4834,-260,7.9935,-263,2.870,-22,4.8490,-36,...,-250,-148,-235,80,-127,-83,-286,-28,13,-164


In [417]:
X = df.drop('Result', axis=1)
y = df['Result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [418]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [419]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [420]:
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
    # select features using threshold
     selection = SelectFromModel(model, threshold=thresh, prefit=True)
     select_X_train = selection.transform(X_train)
     # train model
     selection_model = xgb.XGBClassifier()
     selection_model.fit(select_X_train, y_train)
     # eval model
     select_X_test = selection.transform(X_test)
     y_pred = selection_model.predict(select_X_test)
     accuracy = accuracy_score(y_test, y_pred)
     print("Thresh=%.6f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

Thresh=0.000000, n=93, Accuracy: 62.96%
Thresh=0.000000, n=93, Accuracy: 62.96%
Thresh=0.000000, n=93, Accuracy: 62.96%
Thresh=0.000000, n=93, Accuracy: 62.96%
Thresh=0.000000, n=93, Accuracy: 62.96%
Thresh=0.000000, n=93, Accuracy: 62.96%
Thresh=0.001182, n=87, Accuracy: 62.96%
Thresh=0.001324, n=86, Accuracy: 63.49%
Thresh=0.001838, n=85, Accuracy: 61.90%
Thresh=0.002432, n=84, Accuracy: 66.14%
Thresh=0.002566, n=83, Accuracy: 64.55%
Thresh=0.002752, n=82, Accuracy: 64.55%
Thresh=0.003035, n=81, Accuracy: 64.55%
Thresh=0.003419, n=80, Accuracy: 65.08%
Thresh=0.003740, n=79, Accuracy: 62.43%
Thresh=0.003922, n=78, Accuracy: 64.55%
Thresh=0.004033, n=77, Accuracy: 61.90%
Thresh=0.004253, n=76, Accuracy: 61.90%
Thresh=0.004413, n=75, Accuracy: 61.90%
Thresh=0.004719, n=74, Accuracy: 60.85%
Thresh=0.004956, n=73, Accuracy: 62.96%
Thresh=0.005059, n=72, Accuracy: 59.26%
Thresh=0.005334, n=71, Accuracy: 61.90%
Thresh=0.005797, n=70, Accuracy: 62.43%
Thresh=0.006087, n=69, Accuracy: 62.96%


### Round of 64

In [314]:
df1 = pd.read_csv("2024 round 1.csv")
dfstats = pd.read_csv("kenpom barttorvik 2024.csv")

In [315]:
for i in df1.index:
    for j in dfstats.index:
        if df1['Team1'][i] == dfstats['TEAM'][j]:
            arr1 = dfstats.iloc[j, 10:]
            break
            
    for k in dfstats.index:    
        if df1['Team2'][i] == dfstats['TEAM'][k]:
            arr2 = dfstats.iloc[k, 10:]
            break
            
    if i == 0:
        data = pd.DataFrame(np.subtract(arr1, arr2))
    else:
        data[i] = np.subtract(arr1, arr2)

data = data.transpose()

In [316]:
seeddiff = []

for i in df1.index:
    seeddiff.append(df1['Seed1'][i] - df1['Seed2'][i])
        
df1['Seed_Diff'] = pd.Series(seeddiff, index=df1.index)

In [317]:
final = pd.concat([df1, data], axis=1)
final = final.drop(['Year','Region Number', 'Region Name'], axis=1)
final.to_csv('R64 TESTING SET.csv', encoding='utf-8', index=False)
final.head()

,Round,Seed1,Team1,Team2,Seed2,Seed_Diff,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,1,1,Connecticut,Stetson,16,-15,-0.3975,22,-1.7103,72,...,61,-66,-13,115,-97,55,-69,-56,-310,-202
1,1,8,Florida Atlantic,Northwestern,9,-1,4.4316,-205,4.3317,-204,...,-199,173,113,-85,41,109,50,-32,-34,56
2,1,5,San Diego St,UAB,12,-7,-1.6828,109,-1.8151,113,...,105,-79,-9,-88,-59,49,-122,52,-241,-78
3,1,4,Auburn,Yale,13,-9,5.345,-267,5.2661,-269,...,-266,52,-49,-84,-122,-191,-8,-67,-92,-77
4,1,6,BYU,Duquesne,11,-5,3.193,-181,2.7345,-166,...,-163,-268,-124,-69,-229,-117,-50,-177,5,-62


In [318]:
#DATASET1 = 'all rounds testing set.csv'
DATASET1 = 'R64 TESTING SET.csv'
df = pd.read_csv(FULL_DATASET)
df1 = pd.read_csv(DATASET1)

df.head()

,Round,Seed1,Team1,Team2,Seed2,Result,Seed_Diff,K TEMPO,K TEMPO RANK,KADJ T,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,1,1,North Carolina,Mount St Marys,16,1,-15,7.6847,-120,6.6882,...,-150,-195,-91,-49,-197,-139,-181,-180,-35,-268
1,1,2,Tennessee,American,15,1,-13,11.7930,-309,9.9867,...,-300,-167,-106,113,-121,255,-242,-76,-66,-225
2,1,3,Louisville,Boise St,14,1,-11,-5.4011,151,-5.5255,...,141,-67,-116,181,-267,31,-154,32,-177,-206
3,1,4,Washington St,Winthrop,13,1,-9,-4.4331,44,-3.1859,...,33,-200,-172,-78,-143,-282,119,-149,26,-149
4,1,5,Notre Dame,George Mason,12,1,-7,8.4834,-260,7.9935,...,-250,-148,-235,80,-127,-83,-286,-28,13,-164


In [319]:
# Drop team names, seeds
df = df.drop(['Team1', 'Team2', 'GAMES', 'W', 'L', 'Seed1', 'Seed2','Seed_Diff', 'Round'], axis=1)
df1noseeds = df1.drop(['Team1', 'Team2', 'GAMES', 'W', 'L', 'Seed1', 'Seed2', 'Seed_Diff', 'Round'], axis=1)

#df['Round'] = df['Round'].astype('category')
#df1['Round'] = df1['Round'].astype('category')
#df['Seed_Diff'] = df['Seed_Diff'].astype('category')
#df1noseeds['Seed_Diff'] = df1noseeds['Seed_Diff'].astype('category')
# df['Seed2'] = df['Seed2'].astype('category')
# df1['Seed2'] = df1['Seed2'].astype('category')

df.dtypes

Result              int64
K TEMPO           float64
K TEMPO RANK        int64
KADJ T            float64
KADJ T RANK         int64
                   ...   
FT% RANK            int64
OP FT% RANK         int64
PPPO RANK           int64
PPPD RANK           int64
ELITE SOS RANK      int64
Length: 91, dtype: object

In [320]:
# Split dataset into X and Y
X_train = df.iloc[:, df.columns != 'Result']
y_train = df['Result']
X_test = df1noseeds

In [321]:
# Standardize features
scaler = StandardScaler()
#scaler = MinMaxScaler(feature_range=(-10, 10))
#scaler = Normalizer()
#scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print(X_train)

[[ 1.71165624 -0.71920299  1.58300925 ... -1.89097446 -0.05762031
  -1.81177805]
 [ 2.67270046 -2.00619365  2.40203275 ... -0.55887659 -0.40695533
  -1.43400471]
 [-1.34947186  1.12616456 -1.44967451 ...  0.82445581 -1.65780011
  -1.26708161]
 ...
 [-0.60312626  0.87421401 -0.68021205 ...  1.52893064  0.4607478
  -0.0986199 ]
 [-0.18205681  0.27498027 -0.61466037 ...  0.42738818 -1.79302658
   0.18251375]
 [-0.56712482  0.77888137 -0.24680125 ...  1.61859108  0.37059683
   0.77992274]]


In [322]:
#Creating an XGBoost classifier
model = xgb.XGBClassifier(colsample_bytree = 0.782841439035954, gamma = 4.88854225172552, learning_rate = 0.22410830516276206, max_depth = 8, min_child_weight = 9, n_estimators = 679, reg_alpha = 3, reg_lambda = 0.30472594896438965, subsample = 0.45813457564102184)

In [323]:
#Training the model on the training data
model.fit(X_train, y_train)

#Making predictions on the test set
y_pred = model.predict(X_test)
print(y_pred)

[1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 1]


In [324]:
df1 = pd.read_csv(DATASET1)
df1.insert(0, "Prediction", y_pred, True)

In [325]:
print(df1)

    Prediction  Round  Seed1             Team1              Team2  Seed2  \
0            1      1      1       Connecticut            Stetson     16   
1            0      1      8  Florida Atlantic       Northwestern      9   
2            0      1      5      San Diego St                UAB     12   
3            1      1      4            Auburn               Yale     13   
4            0      1      6               BYU           Duquesne     11   
5            1      1      3          Illinois        Morehead St     14   
6            1      1      7     Washington St              Drake     10   
7            1      1      2           Iowa St    South Dakota St     15   
8            1      1      1    North Carolina             Howard     16   
9            0      1      8    Mississippi St        Michigan St      9   
10           1      1      5       Saint Marys       Grand Canyon     12   
11           1      1      4           Alabama         Charleston     13   
12          

In [326]:
score = 0
results = [1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1]
team1 = []
team2 = []
seed1 = []
seed2 = []

In [327]:
for i in df1.index:
    if i%2 == 0:
        if y_pred[i] == 1:
            team1.append(df1['Team1'][i])
            seed1.append(df1['Seed1'][i])
        else:
            team1.append(df1['Team2'][i])
            seed1.append(df1['Seed2'][i])
            
    else:    
        if y_pred[i] == 1:
            team2.append(df1['Team1'][i])
            seed2.append(df1['Seed1'][i])
        else:
            team2.append(df1['Team2'][i])
            seed2.append(df1['Seed2'][i])

r32 = zip(seed1, team1, team2, seed2)            
df2 = pd.DataFrame(r32, columns=['Seed1', 'Team1', 'Team2', 'Seed2'])
df2.head()

,Seed1,Team1,Team2,Seed2
0,1,Connecticut,Northwestern,9
1,12,UAB,Auburn,4
2,11,Duquesne,Illinois,3
3,7,Washington St,Iowa St,2
4,1,North Carolina,Michigan St,9


In [328]:
for i in df1.index:
    if df1['Prediction'][i] == results[i]:
        score = score + 10
        
print(score)

240


###

In [329]:
# param_grid = {
#     'learning_rate': [0.01, 0.1, 0.3],
#     'alpha': [0.1, 0.01, 0.001],
#     'lambda': [0.1, 0.01, 0.001],
#     'n_estimators': [100, 500, 1000]
# }

# xgb_model = xgb.XGBClassifier()

# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# print("Best parameters found: ", grid_search.best_params_)

In [330]:
# model_new = xgb.XGBClassifier(max_depth = 3, subsample = 0.8, colsample_bytree = 0.8, learning_rate = 0.1, n_estimators = 600)
# model_new.fit(X_train, y_train)
# y_pred = model_new.predict(X_test)
# df1['Prediction'] = y_pred

# accuracy = accuracy_score(results, y_pred)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [331]:
# from numpy import sort
# from sklearn.feature_selection import SelectFromModel

# # Fit model using each importance as a threshold
# thresholds = sort(model.feature_importances_)
# for thresh in thresholds:
#     # select features using threshold
#      selection = SelectFromModel(model, threshold=thresh, prefit=True)
#      select_X_train = selection.transform(X_train)
#      # train model
#      selection_model = xgb.XGBClassifier()
#      selection_model.fit(select_X_train, y_train)
#      # eval model
#      select_X_test = selection.transform(X_test)
#      y_pred = selection_model.predict(select_X_test)
#      accuracy = accuracy_score(results, y_pred)
#      print("Thresh=%.6f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

In [332]:
# selection = SelectFromModel(model, threshold=0.007545, prefit=True)
# select_X_train = selection.transform(X_train)
# # train model
# selection_model = xgb.XGBClassifier()
# selection_model.fit(select_X_train, y_train)
# # eval model
# select_X_test = selection.transform(X_test)
# y_pred = selection_model.predict(select_X_test)
# accuracy = accuracy_score(results, y_pred)
# print("Thresh=%.6f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

# df1['Prediction'] = y_pred
# print(df1)

### Round of 32

In [333]:
for i in df2.index:
    for j in dfstats.index:
        if df2['Team1'][i] == dfstats['TEAM'][j]:
            arr1 = dfstats.iloc[j, 10:]
            break
            
    for k in dfstats.index:    
        if df2['Team2'][i] == dfstats['TEAM'][k]:
            arr2 = dfstats.iloc[k, 10:]
            break
            
    if i == 0:
        data = pd.DataFrame(np.subtract(arr1, arr2))
    else:
        data[i] = np.subtract(arr1, arr2)

data = data.transpose()

In [334]:
seeddiff = []
for i in df2.index:
    seeddiff.append(df2['Seed1'][i] - df2['Seed2'][i])
df2['Seed_Diff'] = pd.Series(seeddiff, index=df2.index)

round = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
df2['Round'] = pd.Series(round, index=df2.index)

final = pd.concat([df2, data], axis=1)
final.to_csv('R32 TESTING SET.csv', encoding='utf-8', index=False)
final.head()

,Seed1,Team1,Team2,Seed2,Seed_Diff,Round,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,1,Connecticut,Northwestern,9,-8,2,1.6203,-47,0.7329,-13,...,-18,-80,-82,194,-49,16,13,-45,-164,-21
1,12,UAB,Auburn,4,8,2,-2.2464,103,-1.989,95,...,99,100,25,-8,58,29,-83,47,266,55
2,11,Duquesne,Illinois,3,8,2,-3.7182,205,-3.565,195,...,197,278,69,137,324,140,-56,187,-100,74
3,7,Washington St,Iowa St,2,5,2,-1.4924,76,-2.5193,137,...,144,-100,-95,28,32,-13,142,7,45,58
4,1,North Carolina,Michigan St,9,-8,2,5.0637,-247,5.2588,-264,...,-257,10,-22,-31,-12,-199,170,-63,-6,30


In [335]:
DATASET2 = 'R32 TESTING SET.csv'
df2 = pd.read_csv(DATASET2)

In [336]:
# Drop team names, seeds
df2noseeds = df2.drop(['Team1', 'Team2', 'GAMES', 'W', 'L', 'Seed1', 'Seed2', 'Seed_Diff', 'Round'], axis=1)

#df2noseeds['Seed_Diff'] = df2['Seed_Diff'].astype('category')
#df2noseeds['Round'] = df2['Round'].astype('category')

df2noseeds.dtypes

K TEMPO           float64
K TEMPO RANK        int64
KADJ T            float64
KADJ T RANK         int64
K OFF             float64
                   ...   
FT% RANK            int64
OP FT% RANK         int64
PPPO RANK           int64
PPPD RANK           int64
ELITE SOS RANK      int64
Length: 90, dtype: object

In [337]:
# Split dataset into X and Y
X_test = df2noseeds

# Standardize features
X_test = scaler.fit_transform(X_test)

print(X_test)

[[ 0.71881665 -0.57011462  0.51227703 ... -0.74425968 -1.14109534
  -1.12529194]
 [-0.1990364   0.31550032 -0.14551881 ...  0.58529159  2.31890286
   0.06251622]
 [-0.54840307  0.91771849 -0.52638741 ...  2.60852177 -0.62611886
   0.35946826]
 ...
 [ 0.37763937 -0.32804654  0.37223048 ... -2.00155272  0.23485744
   0.5157588 ]
 [ 0.48813685 -0.59373102  0.36522212 ...  1.0477442   0.94295009
  -0.10940338]
 [-0.23433387  0.48081511 -0.15820638 ...  0.00722582  0.91076406
  -0.75019463]]


In [338]:
#Making predictions on the test set
y_pred = model.predict(X_test)
print(y_pred)

[1 0 0 1 1 0 1 0 1 0 1 0 1 1 1 1]


In [339]:
df2.insert(0, "Prediction", y_pred, True)
print(df2)

    Prediction  Seed1           Team1         Team2  Seed2  Seed_Diff  Round  \
0            1      1     Connecticut  Northwestern      9         -8      2   
1            0     12             UAB        Auburn      4          8      2   
2            0     11        Duquesne      Illinois      3          8      2   
3            1      7   Washington St       Iowa St      2          5      2   
4            1      1  North Carolina   Michigan St      9         -8      2   
5            0      5     Saint Marys       Alabama      4          1      2   
6            1     11      New Mexico        Baylor      3          8      2   
7            0     10          Nevada       Arizona      2          8      2   
8            1      1         Houston     Texas A&M      9         -8      2   
9            0     12   James Madison          Duke      4          8      2   
10           1      6      Texas Tech      Kentucky      3          3      2   
11           0     10        Colorado   

In [340]:
results1 = ['Connecticut', 'Illinois', 'North Carolina', 'Clemson', 'Houston', 'North Carolina St', 'Purdue', 'Creighton']
results2 = ['San Diego St', 'Iowa St', 'Alabama', 'Arizona', 'Duke', 'Marquette', 'Gonzaga', 'Tennessee']
team1 = []
team2 = []
seed1 = []
seed2 = []

for i in df2.index:
    if i%2 == 0:
        if y_pred[i] == 1:
            team1.append(df2['Team1'][i])
            seed1.append(df2['Seed1'][i])
        else:
            team1.append(df2['Team2'][i])
            seed1.append(df2['Seed2'][i])
            
    else:    
        if y_pred[i] == 1:
            team2.append(df2['Team1'][i])
            seed2.append(df2['Seed1'][i])
        else:
            team2.append(df2['Team2'][i])
            seed2.append(df2['Seed2'][i])

r16 = zip(seed1, team1, team2, seed2)            
df3 = pd.DataFrame(r16, columns=['Seed1', 'Team1', 'Team2', 'Seed2'])
df3.head()

,Seed1,Team1,Team2,Seed2
0,1,Connecticut,Auburn,4
1,3,Illinois,Washington St,7
2,1,North Carolina,Alabama,4
3,11,New Mexico,Arizona,2
4,1,Houston,Duke,4


In [341]:
i = 0
j = 0
for k in df2.index:
    if df2['Prediction'][k] == 1:
        if k%2 == 0:
            if df2['Team1'][k] == results1[i]:
                score += 20
            i += 1
        else:
            if df2['Team1'][k] == results2[j]:
                score += 20
            j += 1
    else:
        if k%2 == 0:
            if df2['Team2'][k] == results1[i]:
                score += 20
            i += 1
        else:
            if df2['Team2'][k] == results2[j]:
                score += 20
            j += 1
        
print(score)

440


### Sweet 16

In [342]:
for i in df3.index:
    for j in dfstats.index:
        if df3['Team1'][i] == dfstats['TEAM'][j]:
            arr1 = dfstats.iloc[j, 10:]
            break
            
    for k in dfstats.index:    
        if df3['Team2'][i] == dfstats['TEAM'][k]:
            arr2 = dfstats.iloc[k, 10:]
            break
            
    if i == 0:
        data = pd.DataFrame(np.subtract(arr1, arr2))
    else:
        data[i] = np.subtract(arr1, arr2)

data = data.transpose()

In [343]:
seeddiff = []
for i in df3.index:
    seeddiff.append(df3['Seed1'][i] - df3['Seed2'][i])
df3['Seed_Diff'] = pd.Series(seeddiff, index=df3.index)

round = [3, 3, 3, 3, 3, 3, 3, 3]
df3['Round'] = pd.Series(round, index=df3.index)

final = pd.concat([df3, data], axis=1)
final.to_csv('R16 TESTING SET.csv', encoding='utf-8', index=False)
final.head()

,Seed1,Team1,Team2,Seed2,Seed_Diff,Round,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,1,Connecticut,Auburn,4,-3,3,-4.7313,245,-5.3344,271,...,267,-62,-48,156,-34,32,-83,-11,14,-50
1,3,Illinois,Washington St,7,-4,3,3.8709,-209,5.068,-260,...,-258,6,85,-222,-57,-188,-112,-65,100,-55
2,1,North Carolina,Alabama,4,-3,3,-2.7023,34,-2.1984,32,...,33,131,56,-23,-104,45,4,28,-234,36
3,11,New Mexico,Arizona,2,9,3,-0.9495,11,0.8424,-8,...,-9,144,40,16,33,3,172,34,15,40
4,1,Houston,Duke,4,-3,3,-3.5721,127,-3.4103,117,...,132,311,328,-254,19,112,-27,28,-44,-64


In [344]:
DATASET3 = 'R16 TESTING SET.csv'
df3 = pd.read_csv(DATASET3)

df3noseeds = df3.drop(['Team1', 'Team2', 'GAMES', 'W', 'L', 'Seed1', 'Seed2', 'Seed_Diff', 'Round'], axis=1)

#df3noseeds['Seed_Diff'] = df3['Seed_Diff'].astype('category')
#df3noseeds['Round'] = df3['Round'].astype('category')

X_test = df3noseeds

X_test = scaler.fit_transform(X_test)

In [345]:
#Making predictions on the test set
y_pred = model.predict(X_test)
print(y_pred)

df3.insert(0, "Prediction", y_pred, True)
print(df3)

[1 1 1 1 1 1 1 0]
   Prediction  Seed1           Team1          Team2  Seed2  Seed_Diff  Round  \
0           1      1     Connecticut         Auburn      4         -3      3   
1           1      3        Illinois  Washington St      7         -4      3   
2           1      1  North Carolina        Alabama      4         -3      3   
3           1     11      New Mexico        Arizona      2          9      3   
4           1      1         Houston           Duke      4         -3      3   
5           1      6      Texas Tech      Marquette      2          4      3   
6           1      1          Purdue        Gonzaga      5         -4      3   
7           0     11          Oregon          Texas      7          4      3   

   K TEMPO  K TEMPO RANK  KADJ T  ...  BADJT RANK  AVG HGT RANK  EFF HGT RANK  \
0  -4.7313           245 -5.3344  ...         267           -62           -48   
1   3.8709          -209  5.0680  ...        -258             6            85   
2  -2.7023        

In [346]:
results1 = ['Connecticut', 'Alabama', 'Duke', 'Purdue']
results2 = ['Illinois', 'Clemson', 'North Carolina St', 'Tennessee']
team1 = []
team2 = []
seed1 = []
seed2 = []

for i in df3.index:
    if i%2 == 0:
        if y_pred[i] == 1:
            team1.append(df3['Team1'][i])
            seed1.append(df3['Seed1'][i])
        else:
            team1.append(df3['Team2'][i])
            seed1.append(df3['Seed2'][i])
            
    else:    
        if y_pred[i] == 1:
            team2.append(df3['Team1'][i])
            seed2.append(df3['Seed1'][i])
        else:
            team2.append(df3['Team2'][i])
            seed2.append(df3['Seed2'][i])

r8 = zip(seed1, team1, team2, seed2)            
df4 = pd.DataFrame(r8, columns=['Seed1', 'Team1', 'Team2', 'Seed2'])
df4.head()

,Seed1,Team1,Team2,Seed2
0,1,Connecticut,Illinois,3
1,1,North Carolina,New Mexico,11
2,1,Houston,Texas Tech,6
3,1,Purdue,Texas,7


In [347]:
i = 0
j = 0
for k in df3.index:
    if df3['Prediction'][k] == 1:
        if k%2 == 0:
            if df3['Team1'][k] == results1[i]:
                score += 40
            i += 1
        else:
            if df3['Team1'][k] == results2[j]:
                score += 40
            j += 1
    else:
        if k%2 == 0:
            if df3['Team2'][k] == results1[i]:
                score += 40
            i += 1
        else:
            if df3['Team2'][k] == results2[j]:
                score += 40
            j += 1
        
print(score)

560


### Elite 8

In [348]:
for i in df4.index:
    for j in dfstats.index:
        if df4['Team1'][i] == dfstats['TEAM'][j]:
            arr1 = dfstats.iloc[j, 10:]
            break
            
    for k in dfstats.index:    
        if df4['Team2'][i] == dfstats['TEAM'][k]:
            arr2 = dfstats.iloc[k, 10:]
            break
            
    if i == 0:
        data = pd.DataFrame(np.subtract(arr1, arr2))
    else:
        data[i] = np.subtract(arr1, arr2)

data = data.transpose()

In [349]:
seeddiff = []
for i in df4.index:
    seeddiff.append(df4['Seed1'][i] - df4['Seed2'][i])
df4['Seed_Diff'] = pd.Series(seeddiff, index=df4.index)

round = [4, 4, 4, 4]
df4['Round'] = pd.Series(round, index=df4.index)

final = pd.concat([df4, data], axis=1)
final.to_csv('R8 TESTING SET.csv', encoding='utf-8', index=False)
final.head()

,Seed1,Team1,Team2,Seed2,Seed_Diff,Round,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,1,Connecticut,Illinois,3,-2,4,-4.3182,233,-5.4343,273,...,269,16,-81,235,-3,11,33,-5,-125,-8
1,1,North Carolina,New Mexico,11,-10,4,-0.9663,19,-2.2326,33,...,39,-30,-13,-79,-38,-143,12,-12,0,-44
2,1,Houston,Texas Tech,6,-5,4,-3.0512,93,-3.3727,114,...,104,42,86,-64,-33,271,-123,-14,-117,-18
3,1,Purdue,Texas,7,-6,4,0.7901,-51,0.2176,-21,...,-27,-213,-112,130,-5,137,39,-47,-33,-9


In [350]:
DATASET4 = 'R8 TESTING SET.csv'
df4 = pd.read_csv(DATASET4)

df4noseeds = df4.drop(['Team1', 'Team2', 'GAMES', 'W', 'L', 'Seed1', 'Seed2', 'Seed_Diff', 'Round'], axis=1)

#df4noseeds['Seed_Diff'] = df4['Seed_Diff'].astype('category')
#df4noseeds['Round'] = df4['Round'].astype('category')

X_test = df4noseeds

X_test = scaler.fit_transform(X_test)

In [351]:
#Making predictions on the test set
y_pred = model.predict(X_test)
print(y_pred)

df4.insert(0, "Prediction", y_pred, True)
print(df4)

[0 0 1 1]
   Prediction  Seed1           Team1       Team2  Seed2  Seed_Diff  Round  \
0           0      1     Connecticut    Illinois      3         -2      4   
1           0      1  North Carolina  New Mexico     11        -10      4   
2           1      1         Houston  Texas Tech      6         -5      4   
3           1      1          Purdue       Texas      7         -6      4   

   K TEMPO  K TEMPO RANK  KADJ T  ...  BADJT RANK  AVG HGT RANK  EFF HGT RANK  \
0  -4.3182           233 -5.4343  ...         269            16           -81   
1  -0.9663            19 -2.2326  ...          39           -30           -13   
2  -3.0512            93 -3.3727  ...         104            42            86   
3   0.7901           -51  0.2176  ...         -27          -213          -112   

   EXP RANK  TALENT RANK  FT% RANK  OP FT% RANK  PPPO RANK  PPPD RANK  \
0       235           -3        11           33         -5       -125   
1       -79          -38      -143           12     

In [352]:
results1 = ['Connecticut', 'North Carolina St']
results2 = ['Alabama', 'Purdue']
team1 = []
team2 = []
seed1 = []
seed2 = []

for i in df4.index:
    if i%2 == 0:
        if y_pred[i] == 1:
            team1.append(df4['Team1'][i])
            seed1.append(df4['Seed1'][i])
        else:
            team1.append(df4['Team2'][i])
            seed1.append(df4['Seed2'][i])
            
    else:    
        if y_pred[i] == 1:
            team2.append(df4['Team1'][i])
            seed2.append(df4['Seed1'][i])
        else:
            team2.append(df4['Team2'][i])
            seed2.append(df4['Seed2'][i])

r4 = zip(seed1, team1, team2, seed2)            
df5 = pd.DataFrame(r4, columns=['Seed1', 'Team1', 'Team2', 'Seed2'])
df5.head()

,Seed1,Team1,Team2,Seed2
0,3,Illinois,New Mexico,11
1,1,Houston,Purdue,1


In [353]:
i = 0
j = 0
for k in df4.index:
    if df4['Prediction'][k] == 1:
        if k%2 == 0:
            if df4['Team1'][k] == results1[i]:
                score += 80
            i += 1
        else:
            if df4['Team1'][k] == results2[j]:
                score += 80
            j += 1
    else:
        if k%2 == 0:
            if df4['Team2'][k] == results1[i]:
                score += 80
            i += 1
        else:
            if df4['Team2'][k] == results2[j]:
                score += 80
            j += 1
        
print(score)

640


### Final 4

In [354]:
for i in df5.index:
    for j in dfstats.index:
        if df5['Team1'][i] == dfstats['TEAM'][j]:
            arr1 = dfstats.iloc[j, 10:]
            break
            
    for k in dfstats.index:    
        if df5['Team2'][i] == dfstats['TEAM'][k]:
            arr2 = dfstats.iloc[k, 10:]
            break
            
    if i == 0:
        data = pd.DataFrame(np.subtract(arr1, arr2))
    else:
        data[i] = np.subtract(arr1, arr2)

data = data.transpose()

In [355]:
seeddiff = []
for i in df5.index:
    seeddiff.append(df5['Seed1'][i] - df5['Seed2'][i])
df5['Seed_Diff'] = pd.Series(seeddiff, index=df5.index)

round = [5, 5]
df5['Round'] = pd.Series(round, index=df5.index)

final = pd.concat([df5, data], axis=1)
final.to_csv('R4 TESTING SET.csv', encoding='utf-8', index=False)
final.head()

,Seed1,Team1,Team2,Seed2,Seed_Diff,Round,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,3,Illinois,New Mexico,11,-8,5,-1.8767,44,-2.7917,48,...,49,-186,32,-164,-2,-118,-138,-36,118,-60
1,1,Houston,Purdue,1,0,5,-4.8272,207,-4.1103,173,...,181,300,337,-128,-29,107,-214,32,-66,3


In [356]:
DATASET5 = 'R4 TESTING SET.csv'
df5 = pd.read_csv(DATASET5)

df5noseeds = df5.drop(['Team1', 'Team2', 'GAMES', 'W', 'L', 'Seed1', 'Seed2', 'Seed_Diff', 'Round'], axis=1)

#df5noseeds['Seed_Diff'] = df5['Seed_Diff'].astype('category')
#df5noseeds['Round'] = df5['Round'].astype('category')

X_test = df5noseeds

X_test = scaler.fit_transform(X_test)

In [357]:
#Making predictions on the test set
y_pred = model.predict(X_test)
print(y_pred)

df5.insert(0, "Prediction", y_pred, True)
print(df5)

[1 1]
   Prediction  Seed1     Team1       Team2  Seed2  Seed_Diff  Round  K TEMPO  \
0           1      3  Illinois  New Mexico     11         -8      5  -1.8767   
1           1      1   Houston      Purdue      1          0      5  -4.8272   

   K TEMPO RANK  KADJ T  ...  BADJT RANK  AVG HGT RANK  EFF HGT RANK  \
0            44 -2.7917  ...          49          -186            32   
1           207 -4.1103  ...         181           300           337   

   EXP RANK  TALENT RANK  FT% RANK  OP FT% RANK  PPPO RANK  PPPD RANK  \
0      -164           -2      -118         -138        -36        118   
1      -128          -29       107         -214         32        -66   

   ELITE SOS RANK  
0             -60  
1               3  

[2 rows x 100 columns]


In [358]:
team1 = []
team2 = []
seed1 = []
seed2 = []

for i in df5.index:
    if i%2 == 0:
        if y_pred[i] == 1:
            team1.append(df5['Team1'][i])
            seed1.append(df5['Seed1'][i])
        else:
            team1.append(df5['Team2'][i])
            seed1.append(df5['Seed2'][i])
            
    else:    
        if y_pred[i] == 1:
            team2.append(df5['Team1'][i])
            seed2.append(df5['Seed1'][i])
        else:
            team2.append(df5['Team2'][i])
            seed2.append(df5['Seed2'][i])

r2 = zip(seed1, team1, team2, seed2)            
df6 = pd.DataFrame(r2, columns=['Seed1', 'Team1', 'Team2', 'Seed2'])
df6.head()

,Seed1,Team1,Team2,Seed2
0,3,Illinois,Houston,1


In [359]:
if df6['Team1'][0] == 'Connecticut':
    score += 160
if df6['Team2'][0] == 'Purdue':
    score += 160

print(score)

640


### FINAL

In [360]:
for i in df6.index:
    for j in dfstats.index:
        if df6['Team1'][i] == dfstats['TEAM'][j]:
            arr1 = dfstats.iloc[j, 10:]
            break
            
    for k in dfstats.index:    
        if df6['Team2'][i] == dfstats['TEAM'][k]:
            arr2 = dfstats.iloc[k, 10:]
            break
            
    if i == 0:
        data = pd.DataFrame(np.subtract(arr1, arr2))
    else:
        data[i] = np.subtract(arr1, arr2)

data = data.transpose()

In [361]:
seeddiff = []
for i in df6.index:
    seeddiff.append(df6['Seed1'][i] - df5['Seed2'][i])
df6['Seed_Diff'] = pd.Series(seeddiff, index=df6.index)

round = [6]
df6['Round'] = pd.Series(round, index=df6.index)

final = pd.concat([df6, data], axis=1)
final.head()

,Seed1,Team1,Team2,Seed2,Seed_Diff,Round,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,3,Illinois,Houston,1,-8,6,6.391,-285,6.6205,-292,...,-293,-335,-229,-49,18,-209,163,-30,146,16


In [362]:
finalnoseeds = final.drop(['Team1', 'Team2', 'GAMES', 'W', 'L', 'Seed1', 'Seed2', 'Round', 'Seed_Diff'], axis=1)

#finalnoseeds['Seed_Diff'] = final['Seed_Diff'].astype('category')
#finalnoseeds['Round'] = final['Round'].astype('category')

X_test = finalnoseeds

X_test = scaler.fit_transform(X_test)

In [363]:
#Making predictions on the test set
y_pred = model.predict(X_test)
print(y_pred)

df6.insert(0, "Prediction", y_pred, True)
print(df6)

[1]
   Prediction  Seed1     Team1    Team2  Seed2  Seed_Diff  Round
0           1      3  Illinois  Houston      1         -8      6


In [364]:
if df6['Team1'][0] == 'Connecticut' and y_pred[0] == 1:
    score += 320
    
print(score)

640


### Hyperparameter Tuning

In [366]:
# # import packages for hyperparameters tuning
# from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [367]:
# df = pd.read_csv(FULL_DATASET)
# df = df.drop(['Team1', 'Team2', 'GAMES', 'W', 'L', 'Seed1', 'Seed2', 'Seed_Diff', 'Round'], axis=1)
# #df['Seed_Diff'] = df['Seed_Diff'].astype('category')
# #df['Round'] = df['Round'].astype('category')

# df.head()

In [304]:
# X = df.drop('Result', axis=1)
# y = df['Result']

In [368]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [369]:
# from hyperopt.pyll.base import scope

# space={'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 1)),
#         "learning_rate": hp.uniform("learning_rate", 0.001, 1),
#         'max_depth': scope.int(hp.quniform("max_depth", 1, 10, 1)),
#         'gamma': hp.uniform ('gamma', 1, 9),
#         'reg_alpha' : hp.quniform('reg_alpha', 0, 100, 1),
#         'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
#         'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
#         'min_child_weight' : scope.int(hp.quniform('min_child_weight', 0, 10, 1)),
#         'subsample' : hp.uniform('subsample', 0, 1),
#         'seed': 0
#     }

In [370]:
# def objective(space):
#     clf=xgb.XGBClassifier(**space, enable_categorical=True)
    
#     evaluation = [(X_train, y_train), (X_test, y_test)]
    
#     clf.fit(X_train, y_train,
#             eval_set=evaluation, eval_metric="auc",
#             early_stopping_rounds=10,verbose=False)
    

#     y_pred = clf.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred>0.5)
#     print ("SCORE:", accuracy)
#     return {'loss': -accuracy, 'status': STATUS_OK }

In [371]:
# trials = Trials()

# best_hyperparams = fmin(fn = objective,
#                         space = space,
#                         algo = tpe.suggest,
#                         max_evals = 100,
#                         trials = trials)

In [372]:
# print(best_hyperparams)

In [373]:
# from numpy import sort
# from sklearn.feature_selection import SelectFromModel

# model = xgb.XGBClassifier(best_hyperparams, enable_categorical=True)
# model.fit(X_train, y_train)
# # Fit model using each importance as a threshold
# thresholds = sort(model.feature_importances_)
# for thresh in thresholds:
#     # select features using threshold
#      selection = SelectFromModel(model, threshold=thresh, prefit=True)
#      select_X_train = selection.transform(X_train)
#      # train model
#      selection_model = xgb.XGBClassifier()
#      selection_model.fit(select_X_train, y_train)
#      # eval model
#      select_X_test = selection.transform(X_test)
#      y_pred = selection_model.predict(select_X_test)
#      accuracy = accuracy_score(y_test, y_pred)
#      print("Thresh=%.6f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

In [374]:
# print(best_hyperparams)

In [375]:
# print(best_hyperparams)

### First Four

In [31]:
DATASET6 = 'F4 TESTING SET.csv'
df6 = pd.read_csv(DATASET6)
df6.head()

,Round,Seed1,Team1,Team2,Seed2,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,1,10,Colorado St,Virginia,10,5.2295,-80,5.8921,-92,9.3220,...,-94,200,210,-279,98,-300,-22,-189,112,-7
1,1,10,Colorado,Boise St,10,2.4845,-141,0.7015,-56,2.4400,...,-75,-15,-142,-47,-61,-79,-89,-40,13,10
2,1,16,Grambling,Montana St,16,-0.1597,11,-1.2871,69,8.5090,...,48,-188,-205,55,-288,-98,-42,-149,-119,-232
3,1,16,Howard,Wagner,16,6.5981,-230,6.2845,-198,8.9186,...,-200,-19,-35,-180,-268,-184,68,-168,200,-8


In [32]:
# Drop player names
df6 = df6.drop(['Team1', 'Team2', 'Round', 'Seed1', 'Seed2'], axis=1)

X_test = df6

X_test = scaler.fit_transform(X_test)

print(X_test)

[[ 0.65092094  0.34131703  0.91569168 -0.23854133  0.71816902 -0.83787781
   0.15104058 -0.29520898  0.84254933  0.49762506  0.6162228   0.16101849
  -0.37207704  0.42771846 -0.51870425  0.03783734  0.74312677 -0.6144677
  -1.69245584 -1.23624508  0.45083482 -0.84307293  0.38706402  1.05107513
   0.14097545  0.7656974  -0.51832106 -0.41872166 -0.99392887  0.15014605
  -0.15014605  0.99392887  0.64165429  1.44687227  1.34613411 -1.65338983
   0.79475876 -1.69580059 -0.18124986 -0.72868458  0.66511894 -1.31877278
   1.31877278  0.6282292  -0.6282292   0.88568239 -1.45432487 -1.60691341
   1.32399774 -1.69077512  1.72786106 -0.41284265  0.73532569  0.825137
  -0.37903261 -0.56502359  0.55286881 -0.2590567   0.20921604  0.6444503
  -0.729504    0.75350529 -0.10178271  0.77668893 -1.02988168  0.19618342
   0.97122494 -0.24495991 -0.24495991  0.97122494  0.36439122 -1.58974645
   1.29519941  1.47704194  0.91624821  1.68359021 -0.29522634  1.0039414
   0.33803877  1.32739456 -1.32509645  0.71

In [33]:
for i in range(0, 10):
    #Creating an XGBoost classifier
    model = xgb.XGBClassifier()

    #Training the model on the training data
    model.fit(X_train, y_train)

    #Making predictions on the test set
    y_pred = model.predict(X_test)
    print(y_pred)

[1 1 1 0]
[1 1 1 0]
[1 1 1 0]
[1 1 1 0]
[1 1 1 0]
[1 1 1 0]
[1 1 1 0]
[1 1 1 0]
[1 1 1 0]
[1 1 1 0]


In [34]:
df6 = pd.read_csv(DATASET6)
df6.insert(0, "Prediction", y_pred, True)
print(df6)

   Prediction  Round  Seed1        Team1       Team2  Seed2  K TEMPO  \
0           1      1     10  Colorado St    Virginia     10   5.2295   
1           1      1     10     Colorado    Boise St     10   2.4845   
2           1      1     16    Grambling  Montana St     16  -0.1597   
3           0      1     16       Howard      Wagner     16   6.5981   

   K TEMPO RANK  KADJ T  KADJ T RANK  ...  BADJT RANK  AVG HGT RANK  \
0           -80  5.8921          -92  ...         -94           200   
1          -141  0.7015          -56  ...         -75           -15   
2            11 -1.2871           69  ...          48          -188   
3          -230  6.2845         -198  ...        -200           -19   

   EFF HGT RANK  EXP RANK  TALENT RANK  FT% RANK  OP FT% RANK  PPPO RANK  \
0           210      -279           98      -300          -22       -189   
1          -142       -47          -61       -79          -89        -40   
2          -205        55         -288       -98       